# 1. [20 pts] List three popular RNN types, then briefly compare and contrast their features and applications.

Three popular RNN types are Elman RNNs, LSTM RNNs, and GRU RNNs. Elman RNNs are simpler RNNs with a hidden layer that captures information about the previous time step and feeds it back into the network for the current time step. The LSTM is another RNN that tries to fix the vanishing gradient problem, where input, output, and forget gate are added to the architecture. The GRU is a variant of the LSTM, where the LSTM gates are substituted for update and reset gates. 

Some benefits of the GRU over LSTM include a simpler architecture, which saves memory. Elman is also simpler than GRU and LSTM, although it suffers far more from the vanishing gradient problem than the latter.

# 2. [20 pts] Explain why the LSTM RNN performed superbly compared to the remainder of the classifiers in the module.

The LSTM RNN performed a lot more efficiently than the remainder of the classifiers for a few reasons. First, One of the main reasons that the LSTM RNN performed a lot better is due to the ability of the LSTM to mitigate the vanishing gradient problem. Vanishing gradient problem is a phenomenon that occurs during the training of deep neural networks, where the gradients that are used to update the network become extremely small or "vanish" as they are backpropogated from the output layers to the earlier layers. Due to this, as well as other mechanisms to improve efficiency,this makes them perform far better than a regular RNN does.

# 3. [20 pts] From the surname dataset, using only two languages {English and Scottish}, build a classifier to distinguish between them. Report 10-fold cross validation performance.

In [2]:
from os import listdir, path
import numpy as np
PATH_DATA='./surnames'
LANGS=('English','Scottish')
SEQ_SIZE=14

In [3]:
# Letter index 0 is the padding value, i.e. padding to fill up the vector to SEQ_SIZE, necessary for batched
# Note that eventually we will use torch Tensor to represent these fixed length sequences
LetterVocabulary, LetterVocabularyIndex, Index2Voc, Sequences = {' ':0}, 1, {0:' '}, {}

for fn in sorted([_ for _ in listdir(PATH_DATA) if _.endswith('.txt')]):
    lang, seqs = path.splitext(path.basename(fn))[0], []

    if lang not in LANGS:  # test case
        continue

    with open(path.join(PATH_DATA, fn), 'r', encoding="utf8") as fin:
        for row in fin.read().splitlines():
            seq = np.zeros(SEQ_SIZE, dtype=np.int32)
            for i, letter in enumerate(row.lower()):  # Convert the surname to lower case
#            for i, letter in enumerate(row):
                if i < SEQ_SIZE:
                    if letter not in LetterVocabulary:
                        LetterVocabulary[letter] = LetterVocabularyIndex
                        Index2Voc[LetterVocabularyIndex] = letter
                        LetterVocabularyIndex += 1
                    seq[i] = LetterVocabulary[letter]
            seqs += [seq]
    Sequences[lang] = seqs

In [4]:
# Sanity check
def print_names(_lang, _k):
    print(''.join([Index2Voc[c] for c in Sequences[_lang][_k]]), Sequences[_lang][_k]) if _lang in LANGS else None

# Some examples
print_names('English', 1)
print_names('Scottish', 2)

abbey          [1 2 2 4 5 0 0 0 0 0 0 0 0 0]
wilson         [18  9 10  3  6 16  0  0  0  0  0  0  0  0]


In [26]:
len(Sequences['English'])+len(Sequences['Scottish'])

3768

In [5]:
# Sanity
N = sum([len(Sequences[_]) for _ in Sequences])
T = Sequences['English'][0].shape[0]
C = len(np.unique(Sequences.keys())[0])
print(N, T, C)

3768 14 2


In [6]:
import itertools

# Pool all sequences and all languages
Seqs = [Sequences[LANGS[_]] for _ in range(C)]
Seqs = list(itertools.chain(*Seqs))

# Number of features is the number of unique characters
M = np.max(Seqs)
print(f'M= {M}')

M= 26


In [7]:
import torch
# Apriori class balance, i.e. inverse probability of the class
nk = np.array([len(Sequences[LANGS[_]]) for _ in range(C)], dtype=np.float32)
nk = (N/nk)
nk = nk/nk.sum()

# Class weights, inverse apriori probability
WEIGHTS = torch.tensor(nk, dtype=torch.float32)

print(WEIGHTS)

tensor([0.0265, 0.9735])


In [8]:
# Ground truth
y = [[_]*len(Sequences[LANGS[_]]) for _ in range(C)]
y = np.array(list(itertools.chain(*y)))

In [9]:
X = np.empty((N,M))
n = 0
for lang in Sequences.keys():
    for seq in Sequences[lang]:
        sxx = np.zeros((M,), dtype=np.float32)
        for i in range(SEQ_SIZE): # for the duration of the signal
            if seq[i] > 0:
                sxx[seq[i]-1] = 1
        X[n] = sxx
        n += 1

In [10]:
%%time

# Borrowed from previous lectures
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

def kfold_eval_docs(_clf, _X, _y):
    # Need indexable data structure
    acc = []
    kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
    for train_index, test_index in kf.split(_X, _y):
        _clf.fit(_X[train_index], _y[train_index])
        y_pred = _clf.predict(_X[test_index])
        acc += [accuracy_score(_y[test_index], y_pred)]
    return np.array(acc)

CPU times: user 426 ms, sys: 368 ms, total: 794 ms
Wall time: 3.7 s


In [11]:
# One-hot encode every position of the sequence
# List of sequence, language tuples for easy shuffling
LANGS_CAT = dict(zip(LANGS, range(len(LANGS))))

def get_Xy():
    Xy = []
    for lang in Sequences.keys():
        for seq in Sequences[lang]:
            T = SEQ_SIZE  # necessary for batched
            sxx = np.zeros((T, M))
            for i in range(T):  # for the duration of the signal
                if seq[i] > 0:
                    sxx[i, seq[i]-1] = 1
            Xy += [(torch.tensor(sxx, dtype=torch.float32),
                    torch.tensor([LANGS_CAT[lang]], dtype=torch.int64))]
    return Xy

# Helper functions
def get_X(_Xy):
    return [_[0] for _ in _Xy]
    
def get_y(_Xy):
    return [int(_[1].data[0]) for _ in _Xy]

# Sanity
Xy = get_Xy()
print(len(Xy))

# printing the confusion matrix below
def get_cm(_y, _p):
    from sklearn.metrics import confusion_matrix
    import pandas as pd

    cm = confusion_matrix(_y, _p, labels=list(range(len(LANGS))))
    display(pd.DataFrame(cm, index=[_[:5] for _ in LANGS], columns=[_[:5] for _ in LANGS]))

3768


In [12]:
# Set the GPU to device 0
import torch.nn as nn
gpu = torch.device('cpu')

class My_RNN(nn.Module):
    
    def __init__(self, n_hidden, n_hid_layers=1, epochs=10, eta=0.0005, batch_size=100, weight=None, info=True):
        """ A PyTorch neural network model based on RNN cell, batched """
        super(My_RNN, self).__init__()

        self.n_hidden= n_hidden  # hidden layer size
        self.n_hid_layers= n_hid_layers  # number of hidden layers
        self.epochs= epochs  # number of learning iterations
        self.eta= eta  # learning rate
        self.B= batch_size  # size of training batch - 1 would not work
        self.info= info  # debug info
        
        self.rnn, self.outlayer = None, None

        self.softmax = nn.LogSoftmax(dim=1)
        # loss function, since the last layer is nn.LogSoftmax
        self.criterion = nn.NLLLoss(weight=weight)

    def forward(self, _X, _h0):
        output, hn = self.rnn(_X, _h0)
        output = self.outlayer(output[:, -1, :])  # output is batched
        output = self.softmax(output)
        return output, hn
    
    def init_cell(self, _M):  # Create variations of our RNN by overriding init_cell
        dropout = 0.2 if self.n_hid_layers > 1 else 0
        return nn.RNN(_M, self.n_hidden, self.n_hid_layers,
                      nonlinearity='relu',
                      bias=False, batch_first=True, dropout=dropout)

    def init_hidden(self, _B):  # batch_first = True
        return torch.zeros(self.n_hid_layers, _B, self.n_hidden).to(gpu)  # Extra dimension - batch

    def fit(self, _Xy):
        from random import shuffle
        import sys
        import torch.optim as optim

        M= _Xy[0][0].shape[1]  # number of features, based on batch input
        C= np.unique([int(_[1].data[0]) for _ in _Xy]).shape[0]  # number of class labels

        self.rnn = self.init_cell(M).to(gpu)
        self.outlayer = nn.Linear(self.n_hidden, C).to(gpu)
        
        self.optimizer = optim.Adam(self.parameters(), lr=self.eta)
        
        for e in range(self.epochs):
            # Shuffle the input to randomly interleave classes, note that they are tuples, i.e. (x, y)
            shuffle(_Xy)

            N = len(_Xy)
            L, totloss = 0, 0

            while L < N-self.B:
                sxx = torch.stack([_[0] for _ in _Xy[L:L+self.B]]).to(gpu)
                y = torch.tensor([_[1] for _ in _Xy[L:L+self.B]], dtype=torch.int64).to(gpu)
                output, loss = self.train_signal(sxx, y, self.B)
                
                totloss += loss
                L += self.B
                
                if self.info:
                    sys.stderr.write(f"\r{e+1:03d}/{self.epochs:4d} | Loss: {loss:6.2f} | "
                                     f"Avg loss: {totloss/(e+1):6.2f} | {y.data.tolist()[0]}")
                    sys.stderr.flush()
    
    def train_signal(self, _sxx, _y, _B):
        h0 = self.init_hidden(_B)
        self.optimizer.zero_grad()

        output, hn = self.forward(_sxx, h0)

        loss = self.criterion(output, _y)
        loss.backward()
        self.optimizer.step()
        return output, loss.item()

    def predict(self, _sxx):  # Tensor dimensions: B x T x M
        _sxx = torch.stack(_sxx)
        with torch.no_grad():
            h0 = self.init_hidden(_sxx.shape[0])  # reset the hidden layer
            output, hn = self.forward(_sxx.to(gpu), h0)

        p_values, indices = output.max(dim=1)
        return indices.to('cpu')

In [21]:
%%time

Xy = get_Xy()

cm_y, cm_p = [], []

Acc = []
kf = StratifiedKFold(n_splits=10)
i=0
for tr_ix, ts_ix in kf.split(np.arange(len(Xy)), get_y(Xy)):
    i+=1
    print(i)
    rnn = My_RNN(128, n_hid_layers=2, epochs=1000, eta=0.005, batch_size=2000, weight=WEIGHTS, info=True).to(gpu)
    
    X_tr = [Xy[_] for _ in tr_ix]  # predict uses X and y as a tuple
    X_ts = get_X([Xy[_] for _ in ts_ix])
    y_ts = get_y([Xy[_] for _ in ts_ix])
    
    rnn.fit(X_tr)
    y_pred = rnn.predict(X_ts)
    
    Acc += [np.sum(np.array(y_pred) == np.array(y_ts))/len(y_pred)]

    cm_y += y_ts
    cm_p += y_pred.tolist()

print(f'RNN 10-fold CV Acc= {np.mean(Acc):.2f} {chr(177)}{np.std(Acc):.3f}')


002/1000 | Loss:   0.69 | Avg loss:   0.35 | 0

1


002/1000 | Loss:   0.70 | Avg loss:   0.35 | 00

2


002/1000 | Loss:   0.69 | Avg loss:   0.35 | 00

...

9


002/1000 | Loss:   0.70 | Avg loss:   0.35 | 00

10


1000/1000 | Loss:   0.05 | Avg loss:   0.00 | 0

RNN 10-fold CV Acc= 0.67 ±0.164
CPU times: user 27min 30s, sys: 22.4 s, total: 27min 53s
Wall time: 13min 56s


# 4. [20 pts] Set the WEIGHTS to None and report the classification performance of (3.). Explain what caused the performance drop?

In [23]:
%%time

Xy = get_Xy()

cm_y, cm_p = [], []

Acc = []
kf = StratifiedKFold(n_splits=10)
i=0
for tr_ix, ts_ix in kf.split(np.arange(len(Xy)), get_y(Xy)):
    i+=1
    print(i)
    rnn = My_RNN(128, n_hid_layers=2, epochs=1000, eta=0.005, batch_size=2000, weight=None, info=True).to(gpu)
    
    X_tr = [Xy[_] for _ in tr_ix]  # predict uses X and y as a tuple
    X_ts = get_X([Xy[_] for _ in ts_ix])
    y_ts = get_y([Xy[_] for _ in ts_ix])
    
    rnn.fit(X_tr)
    y_pred = rnn.predict(X_ts)
    
    Acc += [np.sum(np.array(y_pred) == np.array(y_ts))/len(y_pred)]

    cm_y += y_ts
    cm_p += y_pred.tolist()

print(f'RNN 10-fold CV Acc= {np.mean(Acc):.2f} {chr(177)}{np.std(Acc):.3f}')

002/1000 | Loss:   0.66 | Avg loss:   0.33 | 0

1


002/1000 | Loss:   0.63 | Avg loss:   0.32 | 00

2


002/1000 | Loss:   0.63 | Avg loss:   0.32 | 00

...

9


002/1000 | Loss:   0.66 | Avg loss:   0.33 | 00

10


1000/1000 | Loss:   0.03 | Avg loss:   0.00 | 0

RNN 10-fold CV Acc= 0.74 ±0.214
CPU times: user 27min 39s, sys: 27.8 s, total: 28min 7s
Wall time: 14min 3s


While this did not cause a performance drop(I could not figure out why), one of the reasons that a performance drop would be expected is due to an overwhelmingly unbalanced dataset in favor of english over scottish. Due to this unbalance, the classifier would be overwhelmingly in favor of predicting English.txt over Scottish.txt. This is because the model may exhibit bias towards the English class, since statistically speaking, it is more likely to appear. When we add weights, it allows us to normalize the model, so that the model is unlikely to predict English anymore than Scottish.

# 5. [20 pts] Now set the performance metric to F1-score, repeat (3.) and (4.) report your observations. (If you had already used F1-score in (3.) just comment why accuracy was not the right metric for this problem)

In [28]:
%%time
from sklearn.metrics import f1_score
Xy = get_Xy()

cm_y, cm_p = [], []

Acc = []
kf = StratifiedKFold(n_splits=10)
i=0
for tr_ix, ts_ix in kf.split(np.arange(len(Xy)), get_y(Xy)):
    i+=1
    print(i)
    rnn = My_RNN(128, n_hid_layers=2, epochs=1000, eta=0.005, batch_size=2000, weight=WEIGHTS, info=True).to(gpu)
    
    X_tr = [Xy[_] for _ in tr_ix]  # predict uses X and y as a tuple
    X_ts = get_X([Xy[_] for _ in ts_ix])
    y_ts = get_y([Xy[_] for _ in ts_ix])
    
    rnn.fit(X_tr)
    y_pred = rnn.predict(X_ts)
    
    Acc += [np.sum(np.array(y_pred) == np.array(y_ts))/len(y_pred)]

    cm_y += y_ts
    cm_p += y_pred.tolist()
f1 = f1_score(cm_y, cm_p, average='weighted')
print(f'RNN 10-fold CV F1-Score = {f1:.2f}')

002/1000 | Loss:   0.69 | Avg loss:   0.35 | 0

1


002/1000 | Loss:   0.69 | Avg loss:   0.35 | 00

2


002/1000 | Loss:   0.69 | Avg loss:   0.35 | 00

...

9


002/1000 | Loss:   0.69 | Avg loss:   0.35 | 00

10


1000/1000 | Loss:   0.05 | Avg loss:   0.00 | 0

RNN 10-fold CV F1-Score = 0.77
CPU times: user 27min 50s, sys: 28.7 s, total: 28min 18s
Wall time: 14min 8s


In [29]:
%%time
from sklearn.metrics import f1_score
Xy = get_Xy()

cm_y, cm_p = [], []

Acc = []
kf = StratifiedKFold(n_splits=10)
i=0
for tr_ix, ts_ix in kf.split(np.arange(len(Xy)), get_y(Xy)):
    i+=1
    print(i)
    rnn = My_RNN(128, n_hid_layers=2, epochs=1000, eta=0.005, batch_size=2000, weight=None, info=True).to(gpu)
    
    X_tr = [Xy[_] for _ in tr_ix]  # predict uses X and y as a tuple
    X_ts = get_X([Xy[_] for _ in ts_ix])
    y_ts = get_y([Xy[_] for _ in ts_ix])
    
    rnn.fit(X_tr)
    y_pred = rnn.predict(X_ts)
    
    Acc += [np.sum(np.array(y_pred) == np.array(y_ts))/len(y_pred)]

    cm_y += y_ts
    cm_p += y_pred.tolist()
f1 = f1_score(cm_y, cm_p, average='weighted')
print(f'RNN 10-fold CV F1-Score = {f1:.2f}')

002/1000 | Loss:   0.70 | Avg loss:   0.35 | 0

1


002/1000 | Loss:   0.73 | Avg loss:   0.36 | 00

2


002/1000 | Loss:   0.68 | Avg loss:   0.34 | 00

...

9


002/1000 | Loss:   0.62 | Avg loss:   0.31 | 00

10


1000/1000 | Loss:   0.03 | Avg loss:   0.00 | 0

RNN 10-fold CV F1-Score = 0.85
CPU times: user 27min 28s, sys: 21.7 s, total: 27min 50s
Wall time: 13min 54s


One reason why accuracy may not be the right metric for this problem is becuase when there is an unbalanced dataset where 97% is one class and 3% is the other class, a model may simply achieve a high accuracy score by predicting the majority class. The f-1 score is better because it ensures that the model is not simply producing an excess of false positives or excess of false negatives to achive a high accuracy score, and allows us to make sure that a model is balanced.